In [1]:
import torch

# Check if PyTorch is installed properly
print("PyTorch version:", torch.__version__)
# Check if CUDA (GPU support) is available
cuda_available = torch.cuda.is_available()
print("CUDA available:", cuda_available)
if cuda_available:
    # Check the GPU device name
    gpu_name = torch.cuda.get_device_name(0)
    print("GPU device name:", gpu_name)
    # Check the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print("Number of available GPUs:", num_gpus)
else:
    print("CUDA is not available. PyTorch is using the CPU.")

PyTorch version: 2.4.0
CUDA available: True
GPU device name: NVIDIA GeForce RTX 4090
Number of available GPUs: 1


In [2]:
import os
import torch
import soundfile as sf
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torch.nn as nn
import torch.optim as optim
import librosa
from tqdm import tqdm
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from torch.utils.data import DataLoader, TensorDataset
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
# # Initialize Wav2Vec2 model and feature extractor
# model_name = "facebook/wav2vec2-large-960h"
# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
# model = Wav2Vec2Model.from_pretrained(model_name)
# min_duration = 0.6  # Minimum duration for padding/truncation

In [5]:
# Move model to the GPU
# model.to(device)

Reading Features

In [8]:
# W2V
# XLSR
# Set paths to feature files
data_dir = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\Features\\SSL"
data_dir1 = "F:\\Awais_data\\Datasets\\ASV21\\Features\\SSL\\W2V\\LA\\"

X_train_hidden_file = os.path.join(data_dir, "W2V_Train_hidden_states_features.npy")
X_val_hidden_file = os.path.join(data_dir, "W2V_dev_hidden_states_features.npy")
# X_test_hidden_file = os.path.join(data_dir, "W2V_eval_hidden_states_features.npy")
X_test_hidden_file = os.path.join(data_dir1, "W2V_LA_hidden_states_features01.npy")

# X_train_cnn_file = os.path.join(data_dir, "XLSR_Train_features_last_cnn_layer.npy")
# X_val_cnn_file = os.path.join(data_dir, "XLSR_dev_features_last_cnn_layer.npy")
# X_test_cnn_file = os.path.join(data_dir, "XLSR_eval_features_last_cnn_layer.npy")
y_train_file = os.path.join(data_dir, "W2V_Train_labels.npy")
y_val_file = os.path.join(data_dir, "W2V_dev_labels.npy")
y_test_file = os.path.join(data_dir1, "W2V_LA_labels01.npy")

In [9]:
# Load features and labels
def load_features(file_path):
    return np.load(file_path)

# Load features and labels with progress bar
print("Loading features and labels...")
with tqdm(total=6) as pbar:
    X_train_hidden = torch.tensor(load_features(X_train_hidden_file), device=device, dtype=torch.float32)
    pbar.update(1)
    X_val_hidden = torch.tensor(load_features(X_val_hidden_file), device=device, dtype=torch.float32)
    pbar.update(1)
    X_test_hidden = torch.tensor(load_features(X_test_hidden_file), device=device, dtype=torch.float32)
    pbar.update(1)
    # X_train_cnn = torch.tensor(load_features(X_train_cnn_file), device=device, dtype=torch.float32)
    # pbar.update(1)
    # X_val_cnn = torch.tensor(load_features(X_val_cnn_file), device=device, dtype=torch.float32)
    # pbar.update(1)
    # X_test_cnn = torch.tensor(load_features(X_test_cnn_file), device=device, dtype=torch.float32)
    # pbar.update(1)
    y_train = torch.tensor(load_features(y_train_file), device=device, dtype=torch.float32)
    pbar.update(1)
    y_val = torch.tensor(load_features(y_val_file), device=device, dtype=torch.float32)
    pbar.update(1)
    y_test = torch.tensor(load_features(y_test_file), device=device, dtype=torch.float32)
    pbar.update(1)


Loading features and labels...


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [10:27<00:00, 104.63s/it]


In [11]:
# Print shapes and sizes
print("Training Hidden Features Shape:", X_train_hidden.shape)
print("Validation Hidden Features Shape:", X_val_hidden.shape)
print("Test Hidden Features Shape:", X_test_hidden.shape)
# print("Training CNN Features Shape:", X_train_cnn.shape)
# print("Validation CNN Features Shape:", X_val_cnn.shape)
# print("Test CNN Features Shape:", X_test_cnn.shape)
print("Training Labels Shape:", y_train.shape)
print("Validation Labels Shape:", y_val.shape)
print("Test Labels Shape:", y_test.shape)

Training Hidden Features Shape: torch.Size([25380, 50176])
Validation Hidden Features Shape: torch.Size([24844, 50176])
Test Hidden Features Shape: torch.Size([40000, 203776])
Training Labels Shape: torch.Size([25380])
Validation Labels Shape: torch.Size([24844])
Test Labels Shape: torch.Size([40000])


In [10]:
# Reshape features with progress bar
print("Reshaping features...")
with tqdm(total=3) as pbar:
    X_train_hidden = X_train_hidden.view(X_train_hidden.shape[0], -1)
    pbar.update(1)
    X_val_hidden = X_val_hidden.view(X_val_hidden.shape[0], -1)
    pbar.update(1)
    X_test_hidden = X_test_hidden.view(X_test_hidden.shape[0], -1)
    pbar.update(1)
    # X_train_cnn = X_train_cnn.view(X_train_cnn.shape[0], -1)
    # pbar.update(1)
    # X_val_cnn = X_val_cnn.view(X_val_cnn.shape[0], -1)
    # pbar.update(1)
    # X_test_cnn = X_test_cnn.view(X_test_cnn.shape[0], -1)
    # pbar.update(1)

Reshaping features...


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 239.93it/s]


In [12]:

X_train = X_train_hidden
X_val = X_val_hidden
X_test = X_test_hidden

# X_train = X_train_cnn
# X_val = X_val_cnn
# X_test = X_test_cnn

In [13]:
print("Before normalization")
print(X_train[0])
print(y_train[0])
print(X_val[0])
print(y_val[0])
print(X_test[0])
print(y_test[0])
print("Shapes")
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

Before normalization
tensor([-0.1370, -0.0350, -0.0693,  ..., -1.1063,  0.6121,  0.1675],
       device='cuda:0')
tensor(0., device='cuda:0')
tensor([ 0.2082,  0.2715, -0.0374,  ..., -0.2409,  0.2648,  0.1941],
       device='cuda:0')
tensor(0., device='cuda:0')
tensor([-0.7414,  0.2916,  0.1293,  ..., -0.7211,  0.9813,  0.2331],
       device='cuda:0')
tensor(1., device='cuda:0')
Shapes
torch.Size([25380, 50176])
torch.Size([25380])
torch.Size([24844, 50176])
torch.Size([24844])
torch.Size([40000, 203776])
torch.Size([40000])


In [15]:
# Function to standardize in batches
def batch_standardize(X, batch_size):
    num_features = X.size(1)
    mean = torch.zeros(num_features, device=device)
    std = torch.zeros(num_features, device=device)
    
    # Compute mean and std in batches
    num_batches = (X.size(0) + batch_size - 1) // batch_size
    for i in range(num_batches):
        batch = X[i*batch_size:(i+1)*batch_size]
        mean += batch.mean(dim=0)
        std += batch.std(dim=0)
    
    mean /= num_batches
    std /= num_batches
    
    # Standardize in batches
    standardized = torch.empty_like(X)
    for i in range(num_batches):
        batch = X[i*batch_size:(i+1)*batch_size]
        standardized[i*batch_size:(i+1)*batch_size] = (batch - mean) / std
    
    return standardized

print("Standardizing features...")
batch_size = 500  # Adjust batch size based on available memory
X_train = batch_standardize(X_train, batch_size)
X_val = batch_standardize(X_val, batch_size)
X_test = batch_standardize(X_test, batch_size)

print("Processing complete.")

Standardizing features...


OutOfMemoryError: CUDA out of memory. Tried to allocate 30.37 GiB. GPU 0 has a total capacity of 23.99 GiB of which 0 bytes is free. Of the allocated memory 49.14 GiB is allocated by PyTorch, and 21.08 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print("After normalization")
print(X_train[0])
print(y_train[0])
print(X_val[0])
print(y_val[0])
print(X_test[0])
print(y_test[0])
print("Shapes")
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# import torch
# import numpy as np
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt

# # Define the visualize_features function
# def visualize_features(X, y, title):
#     X_embedded = TSNE(n_components=2).fit_transform(X.cpu())
#     plt.figure(figsize=(10, 8))
#     scatter = plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=y.cpu(), cmap='viridis', alpha=0.5)
#     plt.colorbar(scatter)
#     plt.title(title)
#     plt.show()

# # Define the function to visualize positive and negative classes separately
# def visualize_class(X, y, class_label, title):
#     X_class = X[y == class_label]
#     X_embedded = TSNE(n_components=2).fit_transform(X_class.cpu())
#     plt.figure(figsize=(10, 8))
#     plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c='green' if class_label == 1 else 'red', alpha=0.5)
#     plt.title(title)
#     plt.show()

# # Load your data as PyTorch tensors
# # For example, assuming the data is already loaded into variables X_train, y_train, X_val, y_val, X_test, y_test

# # Visualize training set
# # visualize_features(X_train, y_train, "Training Set")

# # Visualize validation set
# # visualize_features(X_val, y_val, "Validation Set")

# # Visualize test set
# # visualize_features(X_test, y_test, "Test Set")

# # Visualize combined data
# # X_combined = torch.cat([X_train, X_val, X_test], dim=0)
# # y_combined = torch.cat([y_train, y_val, y_test], dim=0)
# # visualize_features(X_combined, y_combined, "Combined Training, Validation and Testing Set")

# # Visualize positive and negative classes separately for each set

# # Training Set
# visualize_class(X_train, y_train, 1, "Training Set - Positive Class")
# visualize_class(X_train, y_train, 0, "Training Set - Negative Class")

# # Validation Set
# visualize_class(X_val, y_val, 1, "Validation Set - Positive Class")
# visualize_class(X_val, y_val, 0, "Validation Set - Negative Class")

# # Test Set
# visualize_class(X_test, y_test, 1, "Test Set - Positive Class")
# visualize_class(X_test, y_test, 0, "Test Set - Negative Class")

# # Combined Set
# # visualize_class(X_combined, y_combined, 1, "Combined Set - Positive Class")
# visualize_class(X_combined, y_combined, 0, "Combined Set - Negative Class")



MLP Training and Testing

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Define the MLP model
# Define the MLP model
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [ ]:
# Hyperparameters as a configuration dictionary
config = {
    'hidden_dim': 1024,
    'output_dim': 1,  # Assuming binary classification
    'num_epochs': 100,
    'batch_size': 128,
    'learning_rate': 0.0001,
    'model_save_path': 'W2V_best_mlp_model_asv21_hid01'  # Base path for saving the model
}
# Automatically determine input_dim from training data
input_dim = X_train.shape[1]


In [ ]:
# Prepare datasets and dataloaders
def create_dataloader(X, y, batch_size):
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

train_val_loader = create_dataloader(torch.cat((X_train, X_val)), torch.cat((y_train, y_val)), config['batch_size'])
test_loader = create_dataloader(X_test, y_test, config['batch_size'])


In [ ]:
# Initialize model, loss function, and optimizer
model = MLPClassifier(input_dim, config['hidden_dim'], config['output_dim']).cuda()
criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

In [ ]:
# Print model summary
summary(model, (input_dim,))

In [ ]:
# Training function
def train_model(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0
    for X_batch, y_batch in tqdm(dataloader, desc="Training", leave=False):
        X_batch, y_batch = X_batch.cuda(), y_batch.float().cuda()  # Ensure target is float for BCEWithLogitsLoss
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)


# Evaluation function
def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    all_labels = []
    all_outputs = []
    num_batches = len(dataloader)
    
    with torch.no_grad():
        for batch_idx, (X_batch, y_batch) in enumerate(tqdm(dataloader, desc="Evaluation", leave=False)):
            X_batch, y_batch = X_batch.cuda(), y_batch.float().cuda()
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
            all_labels.extend(y_batch.cpu().numpy())
            all_outputs.extend(torch.sigmoid(outputs).cpu().numpy())  # Use sigmoid to convert logits to probabilities

            # Debug information: print batch index and batch size
            if batch_idx % 100 == 0:  # Print every 100 batches
                print(f"Processed batch {batch_idx}/{num_batches}, batch size: {len(X_batch)}")
    
    # Calculate metrics
    all_labels = np.array(all_labels)
    all_outputs = np.array(all_outputs)
    
    # Accuracy
    predictions = [1 if x > 0.5 else 0 for x in all_outputs]
    accuracy = accuracy_score(all_labels, predictions)
    
    # Precision, Recall, F1
    precision = precision_score(all_labels, predictions, zero_division=1)
    recall = recall_score(all_labels, predictions)
    f1 = f1_score(all_labels, predictions)
    
    # AUC
    auc = roc_auc_score(all_labels, all_outputs)
    
    # EER Calculation
    fpr, tpr, thresholds = roc_curve(all_labels, all_outputs)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.abs(fnr - fpr))]
    eer = fpr[np.nanargmin(np.abs(fnr - fpr))]
    
    # Confusion Matrix
    cm = confusion_matrix(all_labels, predictions)
    
    return total_loss / num_batches, accuracy, precision, recall, f1, auc, eer, cm


In [ ]:
epochs_no_improve = 0
n_epochs_stop = 10
best_val_loss = float('inf')
best_val_eer = float('inf')
for epoch in range(config['num_epochs']):
    print(f'Starting epoch {epoch+1}/{config["num_epochs"]}')
    train_loss = train_model(model, train_val_loader, criterion, optimizer)
    
    # Validate using the validation set
    val_loss, val_accuracy, val_precision, val_recall, val_f1, val_auc, val_eer, cm = evaluate_model(model, create_dataloader(X_val, y_val, config['batch_size']), criterion)
    
    print(f'Epoch {epoch+1}/{config["num_epochs"]}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val AUC: {val_auc:.4f}, Val EER: {val_eer:.4f}')
       
    if val_eer < best_val_eer:
        best_val_eer = val_eer
        epochs_no_improve = 0
        
        # Remove the previously saved model if it exists
        if os.path.exists(config['model_save_path']):
            os.remove(config['model_save_path'])
        
        # Save the current model
        torch.save(model.state_dict(), config['model_save_path'])
        print(f'Saved best model with Val EER: {val_eer:.4f} to {config["model_save_path"]}')
    else:
        epochs_no_improve += 1
    
    # Early stopping
    if epochs_no_improve >= n_epochs_stop:
        print(f'Early stopping at epoch {epoch+1}')
        break

        
    # Save the best model
    # if val_loss < best_val_loss:
    #     best_val_loss = val_loss
    #     model_save_path = config['model_save_path_template'].format(val_loss)
    #     torch.save(model.state_dict(), model_save_path)
    #     print(f'Saved best model with Val Loss: {val_loss:.4f} to {model_save_path}')

In [ ]:
pwd

In [ ]:
# After training, load the best model for testing
if os.path.exists(config['model_save_path']):
    model.load_state_dict(torch.load(config['model_save_path']))
    print(f'Loaded best model from {config["model_save_path"]} for testing')

In [ ]:
# Load the best model for testing
best_model_path = config['model_save_path'].format(best_val_loss)
model.load_state_dict(torch.load(best_model_path))

In [ ]:
model.eval()

In [ ]:
# Testing the model
test_loss, test_accuracy, test_precision, test_recall, test_f1, test_auc, test_eer, test_cm = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Precision: {test_precision:.4f}')
print(f'Test Recall: {test_recall:.4f}')
print(f'Test F1: {test_f1:.4f}')
print(f'Test AUC: {test_auc:.4f}')
print(f'Test EER: {test_eer:.4f}')
print(f'Test Confusion Matrix:\n{test_cm}')

In [ ]:
from sklearn.svm import OneClassSVM

# Transform y_train to be 1 for the positive class and -1 for the negative class
y_train_one_class = np.where(y_train == 1, 1, -1)
y_val_one_class = np.where(y_val == 1, 1, -1)
y_test_one_class = np.where(y_test == 1, 1, -1)

one_class_model = OneClassSVM(kernel="rbf", gamma='scale', nu=0.5)
one_class_model.fit(X_train[y_train == 1])  # Train only on the positive class

# Evaluate the model
val_scores = one_class_model.decision_function(X_val)
test_scores = one_class_model.decision_function(X_test)

val_auc = roc_auc_score(y_val_one_class, val_scores)
test_auc = roc_auc_score(y_test_one_class, test_scores)

val_accuracy = accuracy_score(y_val_one_class, np.sign(val_scores))
test_accuracy = accuracy_score(y_test_one_class, np.sign(test_scores))

fpr, tpr, _ = roc_curve(y_test_one_class, test_scores)
test_eer = compute_eer(fpr, tpr)

print(f"One-Class SVM - AUC: {test_auc}, Accuracy: {test_accuracy}, EER: {test_eer}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
from scipy.optimize import brentq
from scipy.interpolate import interp1d

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to process data in batches
def process_in_batches(model, data, batch_size=32):
    model.eval()
    results = []
    with torch.no_grad():
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size].to(device)
            output = model(batch)
            results.append(output.cpu().numpy())
    return np.concatenate(results)

# Transform y_train to be 1 for the positive class and -1 for the negative class
y_train_np = y_train.cpu().numpy()
y_val_np = y_val.cpu().numpy()
y_test_np = y_test.cpu().numpy()

y_train_one_class = np.where(y_train_np == 1, 1, -1)
y_val_one_class = np.where(y_val_np == 1, 1, -1)
y_test_one_class = np.where(y_test_np == 1, 1, -1)

# Define One-Class SVM model
class OneClassSVM(nn.Module):
    def __init__(self, input_dim):
        super(OneClassSVM, self).__init__()
        self.w = nn.Parameter(torch.randn(input_dim, 1))
        self.b = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return x @ self.w + self.b

# Train One-Class SVM only on the positive class
positive_X_train = X_train[y_train == 1].clone().detach().float().to(device)
model = OneClassSVM(input_dim=positive_X_train.shape[1]).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Training loop
epochs = 100
batch_size = 32
for epoch in range(epochs):
    model.train()
    for i in range(0, len(positive_X_train), batch_size):
        batch = positive_X_train[i:i + batch_size]
        optimizer.zero_grad()
        outputs = model(batch)
        target = torch.zeros(outputs.shape).to(device)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

# Evaluate the model using batch processing
val_scores = process_in_batches(model, X_val, batch_size)
test_scores = process_in_batches(model, X_test, batch_size)

val_auc = roc_auc_score(y_val_one_class, val_scores)
test_auc = roc_auc_score(y_test_one_class, test_scores)

val_accuracy = accuracy_score(y_val_one_class, np.sign(val_scores))
test_accuracy = accuracy_score(y_test_one_class, np.sign(test_scores))

def compute_eer(fpr, tpr):
    eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    return eer

fpr, tpr, _ = roc_curve(y_test_one_class, test_scores)
test_eer = compute_eer(fpr, tpr)

print(f"One-Class SVM - AUC: {test_auc}, Accuracy: {test_accuracy}, EER: {test_eer}")
